In [ ]:
# read in functions needed aka Indexer, Vectorising functions, scaling etc

from pyspark.ml.feature import VectorAssembler
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import MinMaxScaler


In [ ]:
#read in data

path = "Datasets/autism-screening-for-toddlders/"
df = spark.read.csv(path+"Toddler Autism dataset July 2018.csv",inferSchema=True,header=True)




In [ ]:
df.limit(6).toPandas()
df.printSchema()  #important to check data types and what
df.groupBy("Class/ASD Traits").count().show(100) #check for class balance

In [ ]:
input_columns = df.columns[1:-1] #take input columns

In [ ]:
dependent_var = "Class/ASD Traits"

renamed = df.withColumn("label_str",df[dependent_var].cast(StringType())
indexer = StringIndexer(inputCol="label_str",outputCol="label")
indexed = indexer.fit(renamed).transform(renamed)
                                                           
                                           

In [ ]:
numeric_inputs = []
string_inputs = []
for column in input_columns:
    if str(indexed.schema[column].dataType) == "StringType":
       indexer = StringIndexer(inputCol=column,outputCol=column+"_num")
       indexed = indexer.fit(indexed).transform(indexed)
       new_col_name = column + "_num"
       string_inputs.append(new_col_name)
    else:
       numeric_inputs.append(column)
    

In [ ]:
d = {}
for col in numeric_inputs:
    d[col] = indexed.approxQuantile(col,[0.01,0.99],0.25 )

for col in numeric_inputs:
    skew = indexed.agg(skewness(indexed[col])).collect()
    skew = skew[0][0]
    if skew > 1:
        indexed = indexed.withColumn(col,\
                  log(when(df[col] < d[col][0],d[col][0]\
                  .when(indexed[col] > d[col][1],dcol[1]\
                  .otherwise(indexed(col)+1).alias(col)
                  print(col,"has been treated for pos skew",skew)
    elif skew < -1:
                        
        indexed = indexed.withColumn(col,\
                  exp(when(df[col] < d[col][0],d[col][0]\
                  .when(indexed[col] > d[col][1],dcol[1]\
                  .otherwise(indexed(col)).alias(col)
                  print(col,"has been treated for neg skew",skew)
        


In [ ]:
features_list = numeric_inputs + string_inputs #combining a single features list which has been indexed

assembler = VectorAssembler(inputCols=features_list,outputCol="features")
output = assembler.transform(indexed).select("features","labels")

In [ ]:
# dealing with negatives, but this scaler doesnt deal well with outliers! use robust scaler

scaler = MinMaxScaler(inputCols="features",outputCol="scaled_features",min=0,max=1000)
scalerModel = scaler.fit(output)